# Left-handedness age-gap analysis & survival analysis examples

This notebook reproduces the Bayes-rule analysis showing how cohort-dependent rates of left-handedness can create an apparent gap in mean age at death, and includes a short survival-analysis example. Run cells sequentially.

## Part A — Bayes-rule reproduction of the left-handed age-gap

Loads digitized left-handedness rates (Gilbert & Wysocki), US death distribution (1999), computes P(A|LH) and P(A|RH) via Bayes' theorem, computes mean ages, and runs Monte Carlo sampling to assess sampling variability.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display


In [ ]:
# Load data (digitized left-handedness rates and US death distribution)
LH_URL = (
    "https://gist.githubusercontent.com/mbonsma/8da0990b71ba9a09f7de395574e54df1/"
    "raw/aec88b30af87fad8d45da7e774223f91dad09e88/lh_data.csv"
)
DEATH_URL = (
    "https://gist.githubusercontent.com/mbonsma/2f4076aab6820ca1807f4e29f75f18ec/"
    "raw/62f3ec07514c7e31f5979beeca86f19991540796/cdc_vs00199_table310.tsv"
)

lh = pd.read_csv(LH_URL)
lh['Birth_year'] = 1986 - lh['Age']
lh['Mean_lh'] = lh[['Female','Male']].mean(axis=1)

deaths = pd.read_csv(DEATH_URL, sep='\t', skiprows=[1])
deaths = deaths.dropna(subset=['Both Sexes']).reset_index(drop=True)
deaths['Age'] = deaths['Age'].astype(int)

print('Left-handedness data (head):')
display(lh.head())
print('\nDeath distribution (head):')
display(deaths.head())


### Plot left-handed rates by birth year

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(lh['Birth_year'], lh['Mean_lh'], marker='o')
plt.xlabel('Birth year')
plt.ylabel('Mean % left-handed')
plt.title('Digitized left-handedness rates (Gilbert & Wysocki, 1986 data)')
plt.grid(True)
import os
OUTDIR = 'outputs'
os.makedirs(OUTDIR, exist_ok=True)
plt.savefig(os.path.join(OUTDIR, 'lh_rates_by_birthyear.png'), dpi=150)
plt.show()


### Plot death distribution (1999)

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(deaths['Age'], deaths['Both Sexes'], marker='o')
plt.xlabel('Age')
plt.ylabel('Number of deaths (Both Sexes)')
plt.title('US deaths by age (1999)')
plt.grid(True)
import os
OUTDIR = 'outputs'
os.makedirs(OUTDIR, exist_ok=True)
plt.savefig(os.path.join(OUTDIR, 'conditional_distributions_1990.png'), dpi=150)
plt.show()


### Functions: P(LH|A), P(LH), P(A|LH), P(A|RH)

In [ ]:
def P_lh_given_A(ages_of_death, lefthanded_data, study_year=1990):
    ages = np.asarray(ages_of_death, dtype=int)
    early_1900s_rate = lefthanded_data['Mean_lh'].iloc[-10:].mean()
    late_1900s_rate = lefthanded_data['Mean_lh'].iloc[:10].mean()
    byears = study_year - ages
    lh_map = lefthanded_data.set_index('Birth_year')['Mean_lh'].to_dict()
    res = np.zeros_like(ages, dtype=float)
    min_by = lefthanded_data['Birth_year'].min()
    max_by = lefthanded_data['Birth_year'].max()
    for i, by in enumerate(byears):
        if by < min_by:
            res[i] = early_1900s_rate / 100.0
        elif by > max_by:
            res[i] = late_1900s_rate / 100.0
        else:
            try:
                res[i] = lh_map[int(by)] / 100.0
            except KeyError:
                nearest = min(lh_map.keys(), key=lambda k: abs(k - by))
                res[i] = lh_map[nearest] / 100.0
    return res


def P_lh(death_distribution, lefthanded_data, study_year=1990):
    ages = death_distribution['Age'].to_numpy(dtype=int)
    p_lh_a = P_lh_given_A(ages, lefthanded_data, study_year)
    weights = death_distribution['Both Sexes'].to_numpy(dtype=float)
    return (weights * p_lh_a).sum() / weights.sum()


def P_A_given_lh(ages, death_distribution, lefthanded_data, study_year=1990):
    ages = np.asarray(ages, dtype=int)
    age_index = death_distribution.set_index('Age')['Both Sexes'].to_dict()
    P_A = np.array([age_index.get(int(a), 0.0) for a in ages], dtype=float)
    if P_A.sum() == 0:
        return np.zeros_like(P_A)
    P_A = P_A / P_A.sum()
    P_left = P_lh(death_distribution, lefthanded_data, study_year)
    P_lh_A = P_lh_given_A(ages, lefthanded_data, study_year)
    with np.errstate(divide='ignore', invalid='ignore'):
        res = (P_lh_A * P_A) / P_left
    res = np.nan_to_num(res)
    if res.sum() > 0:
        res = res / res.sum()
    return res


def P_A_given_rh(ages, death_distribution, lefthanded_data, study_year=1990):
    ages = np.asarray(ages, dtype=int)
    age_index = death_distribution.set_index('Age')['Both Sexes'].to_dict()
    P_A = np.array([age_index.get(int(a), 0.0) for a in ages], dtype=float)
    if P_A.sum() == 0:
        return np.zeros_like(P_A)
    P_A = P_A / P_A.sum()
    P_right = 1.0 - P_lh(death_distribution, lefthanded_data, study_year)
    P_rh_A = 1.0 - P_lh_given_A(ages, lefthanded_data, study_year)
    with np.errstate(divide='ignore', invalid='ignore'):
        res = (P_rh_A * P_A) / P_right
    res = np.nan_to_num(res)
    if res.sum() > 0:
        res = res / res.sum()
    return res


### Compute conditional distributions and mean ages (study year = 1990)

In [ ]:
ages = np.arange(6, 115, 1)
pA_lh = P_A_given_lh(ages, deaths, lh, study_year=1990)
pA_rh = P_A_given_rh(ages, deaths, lh, study_year=1990)
avg_lh = np.nansum(ages * pA_lh)
avg_rh = np.nansum(ages * pA_rh)
print(f"Study year 1990: avg LH = {avg_lh:.4f}, avg RH = {avg_rh:.4f}, gap (LH - RH) = {avg_lh - avg_rh:.4f} years")

plt.figure(figsize=(8,4))
plt.plot(ages, pA_lh, label='P(A|LH)')
plt.plot(ages, pA_rh, label='P(A|RH)')
plt.xlabel('Age at death')
plt.ylabel('Probability')
plt.legend()
plt.title('P(A|LH) vs P(A|RH) (study year=1990)')
import os
OUTDIR = 'outputs'
os.makedirs(OUTDIR, exist_ok=True)
plt.savefig(os.path.join(OUTDIR, 'km_curves.png'), dpi=150)
plt.show()


### Monte Carlo sampling to estimate sampling variability

In [ ]:
def mc_sample_gaps(death_distribution, lefthanded_data, study_year=1990, sample_size=2000, n_sims=2000, random_seed=123):
    rng = np.random.default_rng(random_seed)
    ages_pop = death_distribution['Age'].to_numpy(dtype=int)
    weights = death_distribution['Both Sexes'].to_numpy(dtype=float)
    probs = weights / weights.sum()
    p_lh_by_age = dict(zip(ages_pop, P_lh_given_A(ages_pop, lefthanded_data, study_year)))
    gaps = np.empty(n_sims, dtype=float)
    for i in range(n_sims):
        sampled_ages = rng.choice(ages_pop, size=sample_size, p=probs)
        is_lh = np.array([rng.random() < p_lh_by_age.get(int(a), 0.0) for a in sampled_ages])
        if is_lh.sum() == 0 or (~is_lh).sum() == 0:
            gaps[i] = np.nan
            continue
        lh_mean = sampled_ages[is_lh].mean()
        rh_mean = sampled_ages[~is_lh].mean()
        gaps[i] = lh_mean - rh_mean
    return gaps

# run MC
gaps = mc_sample_gaps(deaths, lh, study_year=1990, sample_size=2000, n_sims=2000, random_seed=123)
print('MC mean gap:', np.nanmean(gaps), 'sd:', np.nanstd(gaps))
plt.figure(figsize=(7,4))
plt.hist(gaps[~np.isnan(gaps)], bins=50)
plt.xlabel('LH mean age - RH mean age (years)')
plt.title('Simulated sampling distribution of mean-age gap (1990)')
import os
OUTDIR = 'outputs'
os.makedirs(OUTDIR, exist_ok=True)
plt.savefig(os.path.join(OUTDIR, 'plot_7.png'), dpi=150)
plt.show()


## Part B — Survival analysis example (Kaplan–Meier & Cox PH)
This example uses a small dataset to demonstrate survival analysis; it is illustrative because individual-level left/right survival data are not available in Part A's datasets.

In [ ]:
# Survival example: toy survival dataset using iris
import pandas as pd
import numpy as np
from lifelines import KaplanMeierFitter, CoxPHFitter
from lifelines.statistics import logrank_test

IRIS_URL = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv'
df = pd.read_csv(IRIS_URL)
np.random.seed(42)
df['time'] = np.random.uniform(1, 10, size=len(df))
df['event'] = np.random.binomial(1, 0.7, size=len(df))
df['group'] = (df['species'] == 'virginica').astype(int)

kmf_a = KaplanMeierFitter()
kmf_b = KaplanMeierFitter()
mask_a = df['group'] == 1
mask_b = df['group'] == 0
T_a = df.loc[mask_a, 'time']; E_a = df.loc[mask_a, 'event']
T_b = df.loc[mask_b, 'time']; E_b = df.loc[mask_b, 'event']
ax = kmf_a.fit(T_a, E_a, label='Group A').plot_survival_function()
kmf_b.fit(T_b, E_b, label='Group B').plot_survival_function(ax=ax)
plt.title('Toy Kaplan-Meier curves')
import os
OUTDIR = 'outputs'
os.makedirs(OUTDIR, exist_ok=True)
plt.savefig(os.path.join(OUTDIR, 'plot_8.png'), dpi=150)
plt.show()

lr = logrank_test(T_a, T_b, event_observed_A=E_a, event_observed_B=E_b)
print('Log-rank p-value:', lr.p_value)

cph_df = df[['time','event','group']].copy()
cph = CoxPHFitter()
cph.fit(cph_df, duration_col='time', event_col='event', formula='group')
cph.print_summary()


### Notes

- Part A reproduces the Bayes-rule explanation showing how cohort effects create an apparent difference in mean age at death between left- and right-handers.
- Part B is illustrative survival analysis using generated survival data.
- Run all cells sequentially in Jupyter. Figures will appear inline.